In [1]:
import csoundengine as ce

In [2]:
e = Engine()
# s = e.session()



--Csound version 6.18 (double samples) Sep 23 2022
[commit: 87da757b874a18322634eb875c5a356f5c94fc23]
libsndfile-1.0.31
sr = 44100.0, kr = 689.062, ksmps = 64
0dBFS level = 1.0, A4 tuning = 442.0
audio buffered in 256 sample-frame blocks
reading 1024-byte blks of shorts from adc:system:capture (RAW)
writing 1024 sample blks of 64-bit floats to dac:system:playback
SECTION 1:


In [3]:
bus = e.assignBus("audio")

In [4]:
e.compile(r'''
instr 100
  ibus = p4
  a0 oscili 0.01, 1000
  busout ibus, a0
endin

instr 200
  ibus = p4
  a0 busin ibus
  outch 1, a0
endin
''')



In [5]:
e.sched(100, 0, 100, [bus])

bususe: bus 0 (token 0), numrefs: 1


100.0002

In [6]:
e.sched(200, 0, 100, [bus])

200.0002

bususe: bus 0 (token 0), numrefs: 2


In [7]:
e.unsched(200)

release. bus 0 (token 0), numrefs before release 2, pull full? 0


In [8]:
e.unsched(100)

release. bus 0 (token 0), numrefs before release 1, pull full? 0
bus 0 returned to pool


In [2]:
import time
import csoundengine as ce
e = ce.Engine()



--Csound version 6.18 (double samples) Sep 23 2022
[commit: 87da757b874a18322634eb875c5a356f5c94fc23]
libsndfile-1.0.31
sr = 44100.0, kr = 689.062, ksmps = 64
0dBFS level = 1.0, A4 tuning = 442.0
audio buffered in 256 sample-frame blocks
reading 1024-byte blks of shorts from adc:system:capture (RAW)
writing 1024 sample blks of 64-bit floats to dac:system:playback
SECTION 1:


In [3]:
e.compile(r'''
instr 110
  ibus = p4
  kfreq line 440, p3, 880
  busout ibus, kfreq
endin

instr 210
  ibus = p4
  kfreq busin ibus
  a0 oscili 0.05, kfreq
  a0 *= linsegr:a(0, 0.1, 1, 0.1, 0)
  outch 1, a0
endin
''')



In [4]:
for x in range(20):
    kbus = e.assignBus("control")
    e.sched(210, 0, 10, args=(kbus,))
    e.sched(110, 0, 10, args=(kbus,))
    print(kbus)
    time.sleep(0.1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
e.unsched(110

In [1]:
from maelzel.core import *

In [2]:
s = playSession()



--Csound version 6.18 (double samples) Sep 23 2022
[commit: 87da757b874a18322634eb875c5a356f5c94fc23]
libsndfile-1.0.31
sr = 44100.0, kr = 689.062, ksmps = 64
0dBFS level = 1.0, A4 tuning = 442.0
audio buffered in 256 sample-frame blocks
reading 1024-byte blks of shorts from adc:system:capture (RAW)
writing 512 sample blks of 64-bit floats to dac:system:playback
SECTION 1:


In [3]:
foo = defPreset('foo', r'''
|ibusL, ibusR|
prints "ibusL: %d, ibusR: %d\n", ibusL, ibusR
aout1 vco2 kamp, kfreq
aout2 vco2 kamp, kfreq+4

busout ibusL, aout1
busout ibusR, aout2
''', routing=False)

In [4]:
busL = s.assignBus()
busR = s.assignBus()

s.defInstr('master', r'''
    |ibusL, ibusR|
    aL busin ibusL
    aR busin ibusR
    outch 1, aL, 2, aR
    ''')

busindex
busassign itoken: 0, ikind: 1
busindex
busassign itoken: 1, ikind: 1


Instr(master, ibusL:5=0, ibusR:6=0)

In [5]:
master = s.sched('master', args=dict(ibusL=busL, ibusR=busR), priority=2)

bus 0 (token 0), numrefs: 1
bus 1 (token 1), numrefs: 1


In [6]:
Note(60.5, 10).play(instr='foo', gain=0.1, ibusL=busL, ibusR=busR)

Button(description='Stop', style=ButtonStyle())

Output()

SynthGroup(n=1)
    Synth(𝍪 preset.foo:501.0002, p4=0 idataidx_:5=17 inumbps:6=2 ibplen:7=3 igain_:8=0.1 ichan:9=1 ipos:10=-1 ifadein:11=0.02 ifadeout:12=0.02 ipchintrp_:13=0 ifadekind_:14=1 ibusL:15=0 ibusR:16=1 ...)

ibusL: 0, ibusR: 1
bus 0 (token 0), numrefs: 2
bus 1 (token 1), numrefs: 2


In [8]:
master.stop()

release. bus 1 (token 1), numrefs before release 1
INIT ERROR in instr 28 (opcode initerror.s) line 544: 
   Bus pool is full!

	initerror.s	"Bus pool is full!"	
		   T 27.773 - note deleted.  i28 had 1 init errors
release. bus 0 (token 0), numrefs before release 1
INIT ERROR in instr 28 (opcode initerror.s) line 544: 
   Bus pool is full!

	initerror.s	"Bus pool is full!"	
		   T 27.773 - note deleted.  i28 had 1 init errors


In [3]:
foo = s.defInstr('foo', r'''
    |itime, Sfoo|
    ielapsed elapsedtime
    prints "Sfoo: %s, itime: %f, ielapsed: %f, diff: %f\n", Sfoo, itime, ielapsed, ielapsed - itime 
    turnoff
''', priority=1)

In [4]:
s.sched("foo", 0, 1, pargs={'Sfoo': 'bar', 'itime': e.elapsedTime()})

Sfoo: bar, itime: 3.826939, ielapsed: 3.831293, diff: 0.004354


Button(description='Stop', style=ButtonStyle())

Output()

Synth(▶ foo:501.0002, p4=0 itime:5=3.82694 Sfoo:6=bar)

Exiting python, closing all active engines
... stopping engine0


end of score.		   overall amps:  0.00000  0.00000  0.00000  0.00000
	   overall samples out of range:        0        0        0        0
0 errors in performance
327646 1024 sample blks of 64-bit floats written to dac:system:playback


In [7]:
e.controlLatency()*2

0.005804988662131519

In [5]:
s.engine._strToIndex

{'cos': 20, 'linear': 21, 'smooth': 22, 'smoother': 23, 'hola': 24}

In [2]:
test = s.defInstr('test', r'''
    |istrid|
    S0 = strget(istrid)
    prints "S0: %s\n", S0
    turnoff
''')

In [3]:
jiji = s.engine.strSet('jiji')

In [4]:
s.sched('test', 0, 1, pargs={'istrid': jiji})

S0: jiji
--- dealloc 501.000200


Synth(■ test:501.0002, p4=0 istrid:5=24)

In [2]:
e = playEngine()
s = e.session()



--Csound version 6.18 (double samples) Aug 23 2022
[commit: a691b9fb3a19ff27df8df4cb44b9c6be0d558fbc]
libsndfile-1.0.31
sr = 44100.0, kr = 689.062, ksmps = 64
0dBFS level = 1.0, A4 tuning = 442.0
audio buffered in 256 sample-frame blocks
reading 1024-byte blks of shorts from adc:system:capture (RAW)
writing 512 sample blks of 64-bit floats to dac:system:playback
SECTION 1:


In [3]:
test = s.defInstr('test', r'''
    |ichan=0|
    prints "ichan: %d\n", ichan
    Schan = "bar"
    ; Schan strget ichan
    prints "Schan: %s\n", Schan
    a0 oscili 0.1, 1000
    ; chnset a0, Schan
''')


In [4]:
print(test.body)

ichan = p5
prints "ichan: %d\n", ichan
Schan = "bar"
; Schan strget ichan
prints "Schan: %s\n", Schan
a0 oscili 0.1, 1000
; chnset a0, Schan


In [16]:
ichan = s.engine.strSet('foo')
ichan

24

In [ ]:
s.sched('test', pargs={'ichan': 0})

In [3]:
e.testAudio()

24.0002


Testaudio: pink noise mode
Channel: 1 / 2
Channel: 2 / 2
Channel: 1 / 2
Channel: 2 / 2


In [6]:
bus = e.assignBus()

In [7]:
bus

1

In [2]:
e = playEngine()
bus = e.assignBus()
bus



--Csound version 6.18 (double samples) Aug 23 2022
[commit: a691b9fb3a19ff27df8df4cb44b9c6be0d558fbc]
libsndfile-1.0.31
sr = 44100.0, kr = 689.062, ksmps = 64
0dBFS level = 1.0, A4 tuning = 442.0
audio buffered in 256 sample-frame blocks
reading 1024-byte blks of shorts from adc:system:capture (RAW)
writing 512 sample blks of 64-bit floats to dac:system:playback
SECTION 1:


0

In [3]:
foo = defPreset('foo', audiogen=r'''
    |ibus=0|
    a1 = oscili(kamp*0.5, kfreq)
    a2 = oscili(kamp*0.5, kfreq+12)
    aout1 = a1+a2
''', out=r'''
    replaceout ichan, aout1
    busout ibus, aout1
    busout 
''')
foo

Preset: foo  
    hasRouting=False
  |ibus=0.0|
  a1 = oscili(kamp*0.5, kfreq)
  a2 = oscili(kamp*0.5, kfreq+12)
  aout = a1+a2
  if ibus >= 0 then
      busout ibus, aout
  else
      outch 1, aout
  endif

In [13]:
print(foo.body)


;5          6       7      8     9     0    1      2      3          4        
idataidx_,inumbps,ibplen,igain_,ichan_,ipos,ifade0,ifade1,ipchintrp_,ifadekind_ passign 5
idatalen_ = inumbps * ibplen
iArgs[] passign idataidx_, idataidx_ + idatalen_
ilastidx = idatalen_ - 1
iTimes[]     slicearray iArgs, 0, ilastidx, ibplen
iPitches[]   slicearray iArgs, 1, ilastidx, ibplen
iAmps[]      slicearray iArgs, 2, ilastidx, ibplen

k_time timeinsts

if ipchintrp_ == 0 then      
    ; linear midi interpolation    
    kpitch, kamp bpf k_time, iTimes, iPitches, iAmps
    kfreq mtof kpitch
elseif (ipchintrp_ == 1) then  ; cos midi interpolation
    kidx bisect k_time, iTimes
    kpitch interp1d kidx, iPitches, "cos"
    kamp interp1d kidx, iAmps, "cos"
    kfreq mtof kpitch
elseif (ipchintrp_ == 2) then  ; linear freq interpolation
    iFreqs[] mtof iPitches
    kfreq, kamp bpf k_time, iTimes, iFreqs, iAmps
    kpitch ftom kfreq
elseif (ipchintrp_ == 3) then  ; cos freq interpolation
    kidx bis

In [10]:
Note(60, 1000).play(instr='foo', fade=0.05, params={'ibus': bus})

Button(description='Stop', style=ButtonStyle())

Output()

SynthGroup(n=1)
    Synth(𝍪 preset.foo:501.0006, p4=0 idataidx_:5=16 inumbps:6=2 ibplen:7=3 igain_:8=1 ichan_:9=1 ipos:10=-1 ifade0:11=0.05 ifade1:12=0.05 ipchintrp_:13=0 ifadekind_:14=1 ibus:15=0 p16=0 ...)

In [7]:
s = e.session()
s.defInstr('master', r'''
    |ibus|
    a0 = busin(ibus)
    outch 1, a0
''')

Instr(master, ibus:5=0)

In [8]:
s.sched('master', priority=2)

Synth(𝍪 master:1001.0002, p4=0 ibus:5=0)

In [12]:
post

Button(description='Stop', style=ButtonStyle())

Output()

Synth(▶ post:2001.0002, p4=0 ibus:5=0)

In [15]:
Note(72, dur=10).play(instr='foo', params=dict(ibus=busnum))


Button(description='Stop', style=ButtonStyle())

Output()

SynthGroup(n=1)
    Synth(𝍪 preset.foo:1001.0002, p4=0 idataidx_:5=16 inumbps:6=2 ibplen:7=3 igain_:8=1 ichan_:9=1 ipos:10=-1 ifade0:11=0.02 ifade1:12=0.02 ipchintrp_:13=0 ifadekind_:14=1 ibus:15=0 p16=0 ...)

Csound tidy up: Segmentation fault
